In [48]:
# load packages
import pandas as pd
import unidecode

In [49]:

arch_file_name = './archeology.json'
raw_df = pd.read_json(arch_file_name)
raw_df.head()

,source,title,infowindow_content,content,address,location,id,custom_filters,infowindow_disable,categories
0,post,Fort Turc de Tamentafoust,"<div class=""fc-main""><div class=""fc-item-title...",,,"{'lat': '36.8074742', 'lng': '3.2305329', 'red...",1597,[],False,[{'icon': 'http://cnra.dz/atlas/wp-content/upl...
1,post,Ruines de l'ancienne cité de Tihamamine,"<div class=""fc-main""><div class=""fc-item-title...",NaN,NaN,"{'lat': '35.9496797', 'lng': '4.549802', 'oncl...",1594,NaN,False,[{'icon': 'http://cnra.dz/atlas/wp-content/upl...
2,post,Fort de Santa-Cruz,"<div class=""fc-main""><div class=""fc-item-title...",NaN,NaN,"{'lat': '35.7098036', 'lng': '-0.6639907', 'on...",1586,NaN,False,[{'icon': 'http://cnra.dz/atlas/wp-content/upl...
3,post,Chapelle de Santa Cruz,"<div class=""fc-main""><div class=""fc-item-title...",NaN,NaN,"{'lat': '35.7098407', 'lng': '-0.664087', 'onc...",1580,NaN,False,[{'icon': 'http://cnra.dz/atlas/wp-content/upl...
4,post,Palais du Bey (Oran),"<div class=""fc-main""><div class=""fc-item-title...",NaN,NaN,"{'lat': '35.7053024', 'lng': '-0.6491188', 'on...",1572,NaN,False,[{'icon': 'http://cnra.dz/atlas/wp-content/upl...


In [50]:
archeology_df = raw_df[['title']]
archeology_df = archeology_df.reindex(columns = ['title', 'lat', 'lon', 'categories', 'permalink', 'wilaya'])  
archeology_df['lat'] = [raw_df.location[i]['lat'] for i in range(0, len(raw_df))]
archeology_df['lon'] = [raw_df.location[i]['lng'] for i in range(0, len(raw_df))]
archeology_df['categories'] = [raw_df.location[i]['extra_fields']['post_categories'] for i in range(0, len(raw_df))]
archeology_df['permalink'] = [raw_df.location[i]['redirect_permalink'] for i in range(0, len(raw_df))]
archeology_df.head()

,title,lat,lon,categories,permalink,wilaya
0,Fort Turc de Tamentafoust,36.8074742,3.2305329,"Alger, Monuments, Ottoman",http://cnra.dz/atlas/fort-turc-de-tamentafoust/,NaN
1,Ruines de l'ancienne cité de Tihamamine,35.9496797,4.549802,"Bordj Bou Arreridj, Médiévale, Sites",http://cnra.dz/atlas/ruines-de-lancienne-cite-...,NaN
2,Fort de Santa-Cruz,35.7098036,-0.6639907,"Oran, Médiévale, Monuments",http://cnra.dz/atlas/fort-de-santa-cruz/,NaN
3,Chapelle de Santa Cruz,35.7098407,-0.664087,"Oran, Médiévale, Monuments",http://cnra.dz/atlas/chapelle-de-santa-cruz/,NaN
4,Palais du Bey (Oran),35.7053024,-0.6491188,"Oran, Monuments, Ottoman",http://cnra.dz/atlas/palais-du-bey-oran/,NaN


In [51]:
# Extract wialya names 
wilayas = archeology_df['categories'].str.split(',')
archeology_df['wilaya'] = [unidecode.unidecode(wilayas[i][0].upper()) for i in range(0, len(wilayas))]
archeology_df.count()

title         186
lat           186
lon           186
categories    186
permalink     186
wilaya        186
dtype: int64

In [52]:
# Add Wilayas codes
# Load Wilayas Features and calculate polygon centroids
wilayas_gjson = '../../dz-admin/wilayas_48.csv'
wilayas_df = pd.read_csv(wilayas_gjson)
wilayas_df = wilayas_df[['code', 'nom_maj']]
wilayas_df = wilayas_df.rename(columns={'nom_maj': 'nom_wil', 'code': 'code_wil'})
wilayas_df.head()

,code_wil,nom_wil
0,1,ADRAR
1,2,CHLEF
2,3,LAGHOUAT
3,4,OUM EL BOUAGHI
4,5,BATNA


In [53]:
# Merge with Wialyas DataFrame
merged = archeology_df.merge(wilayas_df, left_on='wilaya', right_on='nom_wil', how='left')

merged.count()

title         186
lat           186
lon           186
categories    186
permalink     186
wilaya        186
code_wil      186
nom_wil       186
dtype: int64

In [54]:
merged.head()

,title,lat,lon,categories,permalink,wilaya,code_wil,nom_wil
0,Fort Turc de Tamentafoust,36.8074742,3.2305329,"Alger, Monuments, Ottoman",http://cnra.dz/atlas/fort-turc-de-tamentafoust/,ALGER,16,ALGER
1,Ruines de l'ancienne cité de Tihamamine,35.9496797,4.549802,"Bordj Bou Arreridj, Médiévale, Sites",http://cnra.dz/atlas/ruines-de-lancienne-cite-...,BORDJ BOU ARRERIDJ,34,BORDJ BOU ARRERIDJ
2,Fort de Santa-Cruz,35.7098036,-0.6639907,"Oran, Médiévale, Monuments",http://cnra.dz/atlas/fort-de-santa-cruz/,ORAN,31,ORAN
3,Chapelle de Santa Cruz,35.7098407,-0.664087,"Oran, Médiévale, Monuments",http://cnra.dz/atlas/chapelle-de-santa-cruz/,ORAN,31,ORAN
4,Palais du Bey (Oran),35.7053024,-0.6491188,"Oran, Monuments, Ottoman",http://cnra.dz/atlas/palais-du-bey-oran/,ORAN,31,ORAN


In [55]:
pois_gdf = merged[['title','categories', 'permalink', 'code_wil','lat','lon']]

# Sort the DataFrame by code (Wilaya number) and reset the index
pois_gdf = pois_gdf.sort_values(by=['code_wil'],ignore_index=True)
pois_gdf.reset_index(drop=True)
pois_gdf.head()

,title,categories,permalink,code_wil,lat,lon
0,Casbah de Melouka,"Adrar, Médiévale, Monuments",http://cnra.dz/atlas/casbah-de-melouka/,1,27.873400,-0.343033
1,Secteur Sauvegardé -Vieille ville de Tamentit -,"Adrar, Secteurs Sauvegardés",http://cnra.dz/atlas/secteur-sauvegarde-vieill...,1,27.764986,-0.260989
2,Secteur Sauvegardé -Vieille ville de Tenes -,"Chlef, Secteurs Sauvegardés",http://cnra.dz/atlas/secteur-sauvegarde-vieill...,2,36.500613,1.315999
3,Musée de site Ténès,"Chlef, Musées Archéologique",http://cnra.dz/atlas/musee-de-site-tenes/,2,36.512286,1.304813
4,Musée National Public de Chlef,"Chlef, Musées Archéologique",http://cnra.dz/atlas/musee-national-public-de-...,2,36.161463,1.333763


In [56]:
pois_gdf.to_csv('./archeology_to_check.csv', header = True)